<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

elixir を齧る。かじる。

今のイメージ erlang 上で、erlang は 並行処理のためのシステムで、その erlang 上で理想的な言語を作ろうとしたら、ruby + clojure みたいな言語になった。

Dave Thomas と まつもとゆきひろ が勧めているのだからいい言語なのだろう。


* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.html
* https://elixirschool.com/ja/lessons/basics/basics/
* https://elixirschool.com/ja/lessons/basics/control-structures/

In [ ]:
%%capture
!sudo apt install elixir

メモ

このあと `!elixir --version` とするとエラーになるが、以下のコードセルに見るようにとりあえず使えているので先に進む。

`!elixir -h` としたら ワンライナーが使えるらしいことがわかった。

`iex` というのがインタラクティブ環境なのだが、colab では使いにくいのでとりあえず使わない。


In [21]:
!elixir -e 'IO.puts 3 + 3'
!elixir -e 'IO.puts "hello world!"'

6
hello world!


In [22]:
%%writefile temp.exs
IO.puts "this is a pen."

Writing temp.exs


In [23]:
!elixir temp.exs

this is a pen.


ネットで紹介されていた次のコードセルのコードはどうやって実行するのだろう。

説明:

このプログラムでは、Parallel というモジュールに pmap という関数を定義しています。 pmap は、与えられたコレクションに対して map（Ruby での Enumerable#map と同じようなものと考えて下さい）を行なうのですが、 各要素の処理を、要素数の分だけプロセスを生成し、各プロセスで並行に実行する、というものです。 ちょっと見ても、よくわからないような気がしますが、大丈夫、本書を読めば、わかるようになります。

とのこと。

In [ ]:
%%writefile temp.exs
defmodule Parallel do
  def pmap(collection, func) do
    collection
    |> Enum.map(&(Task.async(fn -> func.(&1) end)))
    |> Enum.map(&Task.await/1)
  end
end

コメントは `#`

In [24]:
%%writefile temp.exs
# コメント実験
str = "helloworld!!!!"
IO.puts str

Overwriting temp.exs


In [25]:
!elixir temp.exs

helloworld!!!!


In [30]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!elixir -e 'IO.puts 1000_000_00_0'

15
4095
65535
1000000000


In [33]:
!elixir -e 'IO.puts 1.532e-4'
# .0 とか 1. とかはエラーになる

1.532e-4
** (TokenMissingError) nofile:1: syntax error: expression is incomplete
    (elixir) lib/code.ex:168: Code.eval_string/3



In [36]:
!elixir -e 'IO.puts "日本語が書けますか"'
!elixir -e 'IO.puts "日本語が書けます"'

日本語が書けますか
日本語が書けます


In [50]:
# 関数の括弧の前にスペースはエラーになる
!elixir -e 'IO.puts(0b1111)'
!elixir -e 'IO.puts("にほんご\n日本語")'
!elixir -e "IO.puts('にほんご\n\"日本語\"')"

15
にほんご
日本語
にほんご
"日本語"


In [51]:
# 文字連結
!elixir -e 'IO.puts("ABCD"<>"EFGH")'

ABCDEFGH


値の埋め込み

`#{変数名}` を記述することで、変数の値を埋め込むことができる。


In [52]:
!elixir -e 'val = 1000; IO.puts "val = #{val}"'

val = 1000


真理値

true と false で false と nil が false でそれ以外は true


In [55]:
!elixir -e 'if true do IO.puts "true" end'

true


# いまここ

In [ ]:

●真理値
	Elixirの真理値は「true」と「false」が用意されており、
	falseとnilは偽、それ以外の値は全て真(true)と判断されます。

●アトム
	アトムは名前がそのまま値となる定数です。
	名前の前に「:」をつけることでアトムとして扱われます。

	・例
		iex> :Test == :Test
		true
		iex> :Test == :Test2
		false

	・真理値との関係
		真理値のtrueとfalseの正体はアトムです。

		・証明コード
			iex> true === :true
			true

	・使用できる文字
		アトムに使用できる文字は英字の大文字、小文字、数字、アンダースコア(_)、「@」が
		使用でき、終端文字としてのみ「!」や「?」が使えます。

		・先頭で使える文字
			先頭で使用できる文字は英字とアンダースコア(_)のみです。
			「@」や数字は使えないので注意して下さい。

		・例１
			iex> :yttm@gmail
			:yttm@gmail

		・例２
			iex> :ElixirWolrd!
			:ElixirWolrd!
■演算子
こちらで紹介する演算子は「算術演算子」「比較演算子」「論理演算子」です。
他にも「マッチ演算子」などがありますが、それらは別で説明します。

●算術演算子
	算術演算子は計算を行うための演算子です。
	四則演算を行うための演算子が用意されています。

	・足し算
		足し算を行う場合は「+」演算子を使用します。
		この演算子を二つの値の間に指定すると左辺と右辺の値で足し算が行われます。

		・例
			iex> 1 + 2
			3
			iex> x = 10
			10
			iex> x + 1
			11

	・引き算
		引き算を行う場合は「-」演算子を使用します。
		この演算子を二つの値の間に指定すると左辺と右辺の値で引き算が行われます。

		・例
			iex> 1 - 2
			-1
			iex> x = 10
			10
			iex> x - 1
			9

	・掛け算
		掛け算を行う場合は「*」演算子を使用します。
		この演算子を二つの値の間に指定すると左辺と右辺の値で掛け算が行われます。

		・例
			iex> 5 * 2
			10
			iex> x = 10
			10
			iex> x * 4
			40

	・割り算
		割り算を行う場合は「/」演算子を使用します。
		この演算子を二つの値の間に指定すると左辺と右辺の値で割り算が行われます。

		・例
			iex> 5 / 2
			2.5
			iex> x = 10
			10
			iex> x / 3
			3.3333333333333335

		・強制浮動少数
			Elixirの「/」を使用した割り算は整数同士でも強制的に
			浮動少数として扱われます。

			・例
				iex> 10 / 2
				5.0

			浮動少数ではなく整数としての結果がほしい場合はdiv関数を使用します。

			・例
				iex> div(10, 5)
				2
				iex> div(10, 4)
				2

	・余り算
		割り算の余りを求める余り算を行う場合は「rem」関数を使用します。
		
		・例
			iex> rem(10, 4)
			2
			iex> rem(10, 3)
			1
			iex> rem(4, 2)
			0

●比較演算子
	比較演算子は二つの値を比較する際に使用する演算子です。
	比較の結果が正しければ「true」間違っていれば「false」となります。

	・==
		「==」は左辺と右辺の値が同じ値かを比較する演算子です。
		同じ値なら「true」異なった値なら「false」となります。

		・例
			iex> 1 == 1
			true
			iex> 1 == 2
			false
			iex> x = 10
			10
			iex> y = 10
			10
			iex> x == y
			true

	・!=
		「!=」は左辺と右辺の値が異なっているかを比較する演算子です。
		値が異なっていたら「true」同じなら「false」となります。

		・例
			iex> 1 != 1
			false
			iex> 1 != 2
			true
			iex> str = "Hello"
			"Hello"
			iex> str2 = "Hello Wolrd"
			"Hello Wolrd"
			iex> str != str2
			true
	・===
		「===」はデータ型の違い含めるなど「==」よりも厳密な等号比較が行われます。
		値が同じ値なら「true」異なる値なら「false」となります。

		・例
			iex> 20.0 == 20
			true
			iex> 20.0 === 20
			false

		上の例では１回目の比較は「==」を使用し、20.0と20を比較し、結果はtrueとなっています。
		２回目の比較では「===」を使用し再度20.0と20を比較していますが、
		結果はfalseとなっています。
		これは20.0は浮動少数型、20は整数型なので厳密に比較すると異なっていると
		判断されたためです。

	・!==
		「!==」は「!=」よりも厳密な判定が行われます。
		型などを含めて値が異なっていたら「true」、完全に値が一致していたら「false」となります。

		・例
			iex> 20.0 != 20
			false
			iex> 20.0 !== 20
			true

	・<=
		「<=」は２つ値の大小関係を比較します。
		左辺の値が右辺の値以下の場合「true」左辺の値が右辺の値よりも
		大きければ「false」となります。

		・例
			iex> 20 <= 30
			true
			iex> 20 <= 20
			true
			iex> 20 <= 10
			false

	・<
		「<=」は２つ値の大小関係を比較します。
		左辺の値が右辺の値よりも小さい場合「true」
		左辺の値が右辺の値以上なら「false」となります。

		・例
			iex> 20 < 30
			true
			iex> 20 < 20
			false
			iex> 20 < 10
			false

	・>=
		「>=」は２つ値の大小関係を比較します。
		左辺の値が右辺の値以上の場合「true」左辺の値が右辺の値よりも
		小さければ「false」となります。

		・例
			iex> 30 >= 20
			true
			iex> 30 >= 30
			true
			iex> 20 >= 30
			false

	・>
		「>」は２つ値の大小関係を比較します。
		左辺の値が右辺の値よりも大きい場合「true」
		左辺の値が右辺の値以下なら「false」となります。

		・例
			iex> 30 > 20
			true
			iex> 30 > 30
			false
			iex> 20 > 30
			false

	・文字の比較
		文字列は大文字、小文字を区別して判定します。

		・例
			iex> "ABC" == "ABC"
			true
			iex> "ABC" == "abc"
			false

		上の例で、最初の比較はどちらも「"ABC"」で比較しており、結果はtrueになっています。
		次の比較は右辺が「"ABC"」左辺が「"abc"」で比較しており、結果はfalseです。
		２回目の比較は左辺が全て大文字、右辺が全て小文字なので、別の文字列として扱われ、
		比較の結果としてfalseとなっています。

●論理演算子
	論理演算子は複雑な条件式を行う場合に使用される演算子で、主に式と式の繋ぎとして使用されます。

	・論理和
		論理和は二つの条件式の結果のどちらかがtrueだった場合、
		全体の条件式の結果をtrueとして判定する演算子です。
		「||」記号を使用します。
		
		・例
			iex> "ABC" == "ABC" || 20 == 30
			true
			iex> "ABC" == "abc" || 20 == 30
			false

	・論理積
		論理積は二つの条件式の結果が両方ともtrueだった場合、
		全体の条件式の結果をtrueとして判定する演算子です。
		「&&」記号を使用します。

		・例
			iex> "ABC" == "ABC" && 20 == 20
			true
			iex> "ABC" == "ABC" && 20 == 30
			false
			iex> "ABC" == "def" && 10 > 100
			false

	・否定
		否定は条件式の結果を反転させます。
		条件式の結果がtrueだったらfalse、falseだったらtrueとなります。
		否定は論理積、論理和と異なり否定したい条件式の前に「!」を設定します。
		※否定を行う条件式は()で囲んでください。

		・例
			iex> !("ABC" == "ABC")
			false
			iex> !("ABC" == "DEF")
			true

 

 	 